In [1]:
import psycopg2
from configparser import ConfigParser
import pandas as pd
import random
import numpy as np
import csv

target_dir = "/home/jupyter/notebooks/PoC/data-preparation/output/understanding_data/"

def config(filename='prepare_data.ini', section='phonetic'):
    parser = ConfigParser()
    parser.read(filename)
 
    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))
 
    return db

def db_connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    params = config()
    conn = psycopg2.connect(**params)
    print('Connected to the PostgreSQL database...')
    
    return conn

def read_given_names():
    conn = db_connect()
    sql_result = pd.DataFrame()
    try:
        query = """
            SELECT * FROM (
                SELECT ENG, ARB, SUM(COUNT) AS COUNT FROM (
                    SELECT ENG, ARB, COUNT FROM GIVEN_NAMES_MASTER
                    WHERE ENG IS NOT NULL AND ENG != '' AND ARB IS NOT NULL AND ARB != ''
                ) AS SUB GROUP BY ENG, ARB
                ORDER BY COUNT DESC
            ) AS S
            UNION ALL
            SELECT * FROM (
                SELECT ENG, ARB, SUM(COUNT) AS COUNT FROM (
                    SELECT ENG, ARB, COUNT FROM FAMILY_NAMES_MASTER
                    WHERE ENG IS NOT NULL AND ENG != '' AND ARB IS NOT NULL AND ARB != ''
                ) AS SUB GROUP BY ENG, ARB
                ORDER BY COUNT DESC
            ) AS S
            UNION ALL
            SELECT * FROM (
                SELECT ENG, ARB, SUM(FREQ) AS COUNT FROM (
                    SELECT ENG, ARB, FREQ FROM GIVEN_NAMES_DAN
                    WHERE ENG IS NOT NULL AND ENG != '' AND ARB IS NOT NULL AND ARB != ''
                ) AS SUB GROUP BY ENG, ARB
                ORDER BY COUNT DESC
            ) AS S

            """

        sql_result = pd.read_sql(query, con=conn)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
    return sql_result


def process_given_names_master():
    conn = db_connect()
    sql_result = pd.DataFrame()
    try:
        query = """
            SELECT * FROM (
                SELECT ENG, ARB, SUM(COUNT) AS COUNT FROM (
                    SELECT ENG, ARB, COUNT FROM GIVEN_NAMES_MASTER
                    WHERE ENG IS NOT NULL AND ENG != '' AND ARB IS NOT NULL AND ARB != ''
                ) AS SUB GROUP BY ENG, ARB
                ORDER BY COUNT DESC
            ) AS S
            UNION ALL
            SELECT * FROM (
                SELECT ENG, ARB, SUM(COUNT) AS COUNT FROM (
                    SELECT ENG, ARB, COUNT FROM FAMILY_NAMES_MASTER
                    WHERE ENG IS NOT NULL AND ENG != '' AND ARB IS NOT NULL AND ARB != ''
                ) AS SUB GROUP BY ENG, ARB
                ORDER BY COUNT DESC
            ) AS S
            """

        sql_result = pd.read_sql(query, con=conn)
        
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
    return sql_result

In [2]:
%%time
given_names = process_given_names_master()
given_names['eng'] = given_names['eng'].str.strip()
given_names['arb'] = given_names['arb'].str.strip()

given_names = given_names.sort_values('count', ascending=False).drop_duplicates(subset=['eng', 'arb']).sort_index()
eng_positive_trans = given_names.sort_values('count', ascending=False).drop_duplicates('eng').sort_index()
arb_positive_trans = given_names.sort_values('count', ascending=False).drop_duplicates('arb').sort_index()

del eng_positive_trans['count']
del arb_positive_trans['count']

cols = arb_positive_trans.columns.tolist()
cols = cols[-1:] + cols[:-1]
arb_positive_trans = arb_positive_trans[cols]

eng_positive_trans['similarity'] = 1
arb_positive_trans['similarity'] = 1

eng_possible_variants = list(set(arb_positive_trans['eng'].tolist()))
arb_possible_variants = list(set(eng_positive_trans['arb'].tolist()))

Connected to the PostgreSQL database...
CPU times: user 1.77 s, sys: 81.3 ms, total: 1.85 s
Wall time: 7.45 s


In [3]:
arb_negative_trans = arb_positive_trans.copy()
arb_negative_trans['similarity'] = 0

eng_negative_trans = eng_positive_trans.copy()
eng_negative_trans['similarity'] = 0

In [4]:
%%time
for index, row in arb_negative_trans.iterrows():
    ran_eng = random.sample(eng_possible_variants, 1)
    arb_negative_trans.set_value(index, 'eng', ran_eng[0])

CPU times: user 22.6 s, sys: 10.9 ms, total: 22.6 s
Wall time: 22.6 s


In [ ]:
%%time
for index, row in eng_negative_trans.iterrows():
    ran_arb = random.sample(arb_possible_variants, 1)
    eng_negative_trans.set_value(index, 'arb', ran_arb[0])

In [ ]:
eng_negative_trans

In [5]:
arb_positive_trans[arb_positive_trans['arb'] == 'يحنى']

,arb,eng,similarity
255351,يحنى,YAHNIE,1


In [24]:
%%time
import csv

base_dir = '/home/jupyter/notebooks/PoC/data-preparation/output/transliteration_datasets/'
arb_positive_trans.to_csv(base_dir + 'arb_positive_trans.tsv',sep='\t', quoting=csv.QUOTE_NONE)
eng_positive_trans.to_csv(base_dir + 'eng_positive_trans.tsv',sep='\t', quoting=csv.QUOTE_NONE)
eee.to_csv(base_dir + 'eng_main_ds.tsv',sep='\t', quoting=csv.QUOTE_NONE)
aaa.to_csv(base_dir + 'arb_main_ds.tsv',sep='\t', quoting=csv.QUOTE_NONE)


CPU times: user 2.02 s, sys: 62.5 ms, total: 2.08 s
Wall time: 2.08 s


In [ ]:
arb_positive_trans[arb_positive_trans['arb'].str.contains(' ') | arb_positive_trans['eng'].str.contains(' ')]

In [ ]:
eng_positive_trans[
    eng_positive_trans['arb'].str.contains('|'.join(arabic_not_allowed_characters)) 
    | eng_positive_trans['arb'].str.contains('\[') | eng_positive_trans['arb'].str.contains('\]') 
    | eng_positive_trans['arb'].str.contains('\(') | eng_positive_trans['arb'].str.contains('\)') 
    | eng_positive_trans['eng'].str.contains('|'.join(english_not_allowed_characters)) 
    | eng_positive_trans['eng'].str.contains('\[') | eng_positive_trans['eng'].str.contains('\]') 
    | eng_positive_trans['eng'].str.contains('\(') | eng_positive_trans['eng'].str.contains('\)') 
    | (eng_positive_trans['arb'].str.contains(' ') & ~eng_positive_trans['eng'].str.contains(' ')) 
    | (eng_positive_trans['eng'].str.contains(' ') & ~eng_positive_trans['arb'].str.contains(' '))
]

In [23]:
eee = eng_positive_trans.drop(eng_positive_trans[
    eng_positive_trans['arb'].str.contains('|'.join(arabic_not_allowed_characters)) 
    | eng_positive_trans['arb'].str.contains('\[') | eng_positive_trans['arb'].str.contains('\]') 
    | eng_positive_trans['arb'].str.contains('\(') | eng_positive_trans['arb'].str.contains('\)') 
    | eng_positive_trans['eng'].str.contains('|'.join(english_not_allowed_characters)) 
    | eng_positive_trans['eng'].str.contains('\[') | eng_positive_trans['eng'].str.contains('\]') 
    | eng_positive_trans['eng'].str.contains('\(') | eng_positive_trans['eng'].str.contains('\)') 
    | (eng_positive_trans['arb'].str.contains(' ') & ~eng_positive_trans['eng'].str.contains(' ')) 
    | (eng_positive_trans['eng'].str.contains(' ') & ~eng_positive_trans['arb'].str.contains(' '))
].index).copy()

In [20]:
arb_positive_trans[
    arb_positive_trans['arb'].str.contains('|'.join(arabic_not_allowed_characters)) 
    | arb_positive_trans['arb'].str.contains('\[') | arb_positive_trans['arb'].str.contains('\]') 
    | arb_positive_trans['arb'].str.contains('\(') | arb_positive_trans['arb'].str.contains('\)') 
    | arb_positive_trans['eng'].str.contains('|'.join(english_not_allowed_characters)) 
    | arb_positive_trans['eng'].str.contains('\[') | arb_positive_trans['eng'].str.contains('\]') 
    | arb_positive_trans['eng'].str.contains('\(') | arb_positive_trans['eng'].str.contains('\)') 
    | (arb_positive_trans['arb'].str.contains(' ') & ~arb_positive_trans['eng'].str.contains(' ')) 
    | (arb_positive_trans['eng'].str.contains(' ') & ~arb_positive_trans['arb'].str.contains(' '))
]

,arb,eng,similarity
290,عبد الله,ABDULLAH,1
4340,الله هدايه,HIDAYATULLAH,1
5441,صاحب زادا,SAHIBZADA,1
5824,نواب زادا,NAWABZADA,1
6429,عبد الرزاق,ABDOLRAZAGH,1
6548,سيد مصطفي,SEYEDMOSTAFA,1
6574,محمد رسول,MOHAMMADRASOUL,1
6577,محمد طاها,MOHAMMADTAHA,1
6612,علي محمد,ALIMOHAMMAD,1
6635,امير محمد,AMIRMOHAMMAD,1


In [22]:
aaa = arb_positive_trans.drop(arb_positive_trans[
    arb_positive_trans['arb'].str.contains('|'.join(arabic_not_allowed_characters)) 
    | arb_positive_trans['arb'].str.contains('\[') | arb_positive_trans['arb'].str.contains('\]') 
    | arb_positive_trans['arb'].str.contains('\(') | arb_positive_trans['arb'].str.contains('\)') 
    | arb_positive_trans['eng'].str.contains('|'.join(english_not_allowed_characters)) 
    | arb_positive_trans['eng'].str.contains('\[') | arb_positive_trans['eng'].str.contains('\]') 
    | arb_positive_trans['eng'].str.contains('\(') | arb_positive_trans['eng'].str.contains('\)') 
    | (arb_positive_trans['arb'].str.contains(' ') & ~arb_positive_trans['eng'].str.contains(' ')) 
    | (arb_positive_trans['eng'].str.contains(' ') & ~arb_positive_trans['arb'].str.contains(' '))
].index).copy()

In [ ]:
allowed_arabic_characters = list("ياول'مرسنهبشدكةجفتحعزخثىقصطإأآؤذًِغئ")

In [ ]:
allowed_arabic_characters = list("ياول'مرسنهبشدكةجفتحعزخثىقصطإأآؤذًِغئ")

In [13]:
arabic_not_allowed_characters = list("AINEROLSMHUTDKGBYCVPJZFW-QX/;,'0123456789")

In [12]:
english_not_allowed_characters = list("ياولمرسنهبشدكةجفتحعزخثىقصطإأآؤذًِغئ/;,0123456789")

In [ ]:
english_not_allowed_characters

In [ ]:
arb_positive_trans[
    (arb_positive_trans['arb'].str.len() <= 2) & (arb_positive_trans['eng'].str.len() <= 2)
]

In [ ]:
eng_positive_trans[
    (eng_positive_trans['arb'].str.len() <= 2) & (eng_positive_trans['eng'].str.len() <= 3)
]

In [ ]:
eng_positive_trans[
    (eng_positive_trans['eng'] == 'MR')
]

In [26]:
len(eee)

431331